## Star Trek Dialogue Analysis - Topic Modeling

In [1]:
import json
import pandas as pd
import os
import re
import numpy as np
import logging
from collections import Counter

# Spacy
import spacy

# NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Gensim
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim.models import LdaModel
from pprint import pprint
from gensim.models import CoherenceModel

# Plotting
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nlp = spacy.load("en_core_web_sm")
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\anon\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
path = r'C:\Users\anon\Documents\CareerFoundry\Data Analytics Immersion\6\Data\Altered Data'

In [4]:
# Define main cast variables

main_cast = {
    'tos': ['KIRK', 'SPOCK', 'UHURA', 'CHEKOV', 'SULU', 'CHAPEL', 'COMPUTER', 'MCCOY', 'SCOTT'],
    'tas': ['KIRK', 'SPOCK', 'UHURA', 'CHEKOV', 'SULU', 'CHAPEL', 'COMPUTER', 'MCCOY', 'SCOTT'],
    'tng': ['PICARD', 'RIKER', 'WORF', 'DATA', 'TROI', 'CRUSHER', 'TASHA', 'CHIEF', "O'BRIEN", 'GUINAN', 'LAFORGE', 'PULASKI', 'WESLEY'],
    'ds9': ['SISKO', 'ODO', 'KIRA', 'JAKE', 'QUARK', 'DAX', "O'BRIEN", 'BASHIR', 'WORF', 'EZRI'],
    'voy': ['JANEWAY', 'CHAKOTAY', 'TUVOK', 'PARIS', 'TORRES', 'KIM', 'EMH', 'NEELIX', 'KES', 'SEVEN', 'ICHEB', 'SESKA'],
    'ent': ['ARCHER', 'DEGRA', 'HOSHI', 'PHLOX', 'REED', 'SHRAN', "T'POL", 'TRAVIS', 'TUCKER'],
    'dis': ['BURNHAM', 'SARU', 'VOQ', 'TYLER', 'STAMETS', 'TILLY', 'LORCA', 'CULBER', 'PIKE', 'BOOK', 'NHAN', 'ADIRA', 'GRAY', 'GEORGIOU', 'DETMER', 'OWOSEKUN', "L'RELL", 'SAREK', 'CORNWELL', 'AIRIAM', 'SPOCK'],
    'pic': ['PICARD', 'AGNES', 'DAHJ', 'DATA', 'ELNOR', 'HUGH', 'SOJI', 'RAFFI', 'RIOS', 'NAREK', 'SEVEN', 'RIZZO']
}

### Load in and prepare data

In [5]:
series_dataframes = {}

In [6]:
# Grab series csvs and load them into dfs
def load_series_dataframes(path):

    for filename in os.listdir(path):
        if filename.endswith('csv'):
            series = filename.split('_')[0]

            # Load the df
            df = pd.read_csv(os.path.join(path, filename))

            # Drop the existing index and 'Unnamed: 0' column if they exist
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            df.reset_index(drop=True, inplace=True)

            # Save the df to csv without the index and 'Unnamed: 0' column
            df.to_csv(f'{series}_df_cleaned.csv', index=False)

            # Store df in the series_dataframes dictionary
            series_dataframes[series] = df

    return series_dataframes

In [7]:
loaded_dataframes = load_series_dataframes(path)

In [8]:
for series in series_dataframes.keys():
    print(series)

dis
ds9
ent
pic
tas
tng
tos
voy


In [9]:
# Set view of columns to see the whole column because I like to live dangerously

pd.set_option('max_colwidth', None)

In [10]:
voy_df = series_dataframes.get('voy')

In [11]:
voy_df.head(1)

,Episode,Season,Year,Title,Character,Gender,Dialogue,Flattened Dialogue,Dialogue Length
0,voy_000,voy_s1,1995,"Caretaker, Part I & II",CHAKOTAY,m,"['Damage report.', 'Be creative!', 'Initiating evasive pattern omega. Mark.', ""I need more power if we're going to make it to the Badlands."", ""Tuvok, shut down all the phaser banks. If you can give me another thirty seconds at full impulse, I'll get us into the Badlands."", 'Throw the last photons at them and then give me the power from the torpedo system.', 'Are you reading any plasma storms ahead?', ""That's where I'm going."", 'Hold on!', 'Gul Evek must feel daring today.', 'Can you plot a course through these plasma fields, Mister Tuvok?', 'We can use a time to make some repairs. What was that?', 'Source?', 'Another storm?', ""Anything left in those impulse generators, B'Elanna?"", 'Maximum power.', 'How do you know my name?', ""No. A member of our crew is missing too. B'Elanna Torres, my engineer."", 'Three of us will transport to your ship.', 'Were you going to deliver us into their waiting hands, Vulcan?', 'I see you had help.', 'At least the Vulcan was doing his duty as a Starfleet officer. But you, you betrayed us for what? Freedom from prison? Latinum? What was your price this time?', 'He can tell us where they are.', 'Where are our people?', ""We won't do that."", 'The Ocampa use it for barter?', 'But she got out.', ""Maybe you can't, but we can."", ""If the array is the Ocampa's sole source of energy, why would the Caretaker seal the conduits?"", ""That doesn't necessarily mean he's dying. He could be leaving."", ""My leg is broken. I can't move."", 'Get out of here, Paris, before the whole thing comes down!', ""You get on those stairs, they'll collapse! We'll both die!"", 'Wrong tribe.', ""Fine. Be a fool. If I have to die, at least I'll have the pleasure of watching you go with me."", ""You're too heavy."", ""We've got to get back to our ship."", 'I think so, Captain.', ""Their weapons array has been hit. They're in trouble."", ""I'm setting a collision course, but the guidance system is disabled. I'll have to pilot the ship manually. Get the crew ready to beam to Voyager. And drop shields for transport. Paris, my crew's coming over. Tell one of your crackerjack Starfleet transporter chiefs to keep a lock on me. I'm going to try and take some heat off your tail."", 'Not yet!', 'Wait.', 'Now!', ""They've got me."", ""She's the Captain.""]","Damage report. Be creative! Initiating evasive pattern omega. Mark. I need more power if we're going to make it to the Badlands. Tuvok, shut down all the phaser banks. If you can give me another thirty seconds at full impulse, I'll get us into the Badlands. Throw the last photons at them and then give me the power from the torpedo system. Are you reading any plasma storms ahead? That's where I'm going. Hold on! Gul Evek must feel daring today. Can you plot a course through these plasma fields, Mister Tuvok? We can use a time to make some repairs. What was that? Source? Another storm? Anything left in those impulse generators, B'Elanna? Maximum power. How do you know my name? No. A member of our crew is missing too. B'Elanna Torres, my engineer. Three of us will transport to your ship. Were you going to deliver us into their waiting hands, Vulcan? I see you had help. At least the Vulcan was doing his duty as a Starfleet officer. But you, you betrayed us for what? Freedom from prison? Latinum? What was your price this time? He can tell us where they are. Where are our people? We won't do that. The Ocampa use it for barter? But she got out. Maybe you can't, but we can. If the array is the Ocampa's sole source of energy, why would the Caretaker seal the conduits? That doesn't necessarily mean he's dying. He could be leaving. My leg is broken. I can't move. Get out of here, Paris, before the whole thing comes down! You get on those stairs, they'll collapse! We'll both die! Wrong tribe. Fine. Be a fool. If I have to die, at least I'll h

In [12]:
tng_df = series_dataframes.get('tng')

In [13]:
tng_df.head(1)

Episode  Season  Year                  Title Character Gender  \
0  tng_000  tng_s1  1987  Encounter at Farpoint    PICARD      m   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

### Gensim topic model test Janeway vs Picard all seasons

#### Prep data

In [14]:
# GET UR DATA FOOL: Filter df for rows where 'Character' is 'PICARD' and 'JANEWAY'
# ***Need to figure out how to further parameterize this

picard_df = tng_df[tng_df['Character'] == 'PICARD']

janeway_df = voy_df[voy_df['Character'] == 'JANEWAY']

In [15]:
picard_df

Episode  Season  Year                         Title Character Gender  \
0     tng_000  tng_s1  1987         Encounter at Farpoint    PICARD      m   
11    tng_001  tng_s1  1987                 The Naked Now    PICARD      m   
21    tng_002  tng_s1  1987                 Code of Honor    PICARD      m   
29    tng_003  tng_s1  1987              The Last Outpost    PICARD      m   
36    tng_004  tng_s1  1987  Where No One Has Gone Before    PICARD      m   
...       ...     ...   ...                           ...       ...    ...   
1359  tng_171  tng_s7  1994                     Firstborn    PICARD      m   
1365  tng_172  tng_s7  1994                    Bloodlines    PICARD      m   
1373  tng_173  tng_s7  1994                     Emergence    PICARD      m   
1382  tng_174  tng_s7  1994             Preemptive Strike    PICARD      m   
1388  tng_175  tng_s7  1994            All Good Things...    PICARD      m   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [16]:
picard_df['Flattened Dialogue'].dtype

dtype('O')

In [17]:
# Helper function to tokenize sentences

def make_list_of_words(sentences):
    
    for sentence in sentences:
    
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)

In [18]:
# Make sure flat dialogue is a string so it can be concatenated into one big...thing...

def prep_dialogues(df, column_name='Flattened Dialogue', split_char=". "):
    
    concatenated_string = ' '.join(df[column_name].astype(str))
    
    split_string = concatenated_string.split(split_char)
    
    tokenized_dialogue = list(make_list_of_words(split_string))
    
    new_df = pd.DataFrame({'Tokenized Dialogue': [tokenized_dialogue]})
    
    return new_df

In [19]:
tokenized_picard_df = prep_dialogues(picard_df)

tokenized_janeway_df = prep_dialogues(janeway_df)

In [20]:
tokenized_janeway_df

Tokenized Dialogue
0  [[tom, paris, kathryn, janeway], [served, with, your, father, on, the, al, batani], [wonder, if, we, could, go, somewhere, and, talk], [about, job, we, like, you, to, do, for, us], [ve, been, told, the, rehab, commission, is, very, pleased, with, your, work], [they, ve, given, me, their, approval, to, discuss, this, matter, with, you], [your, father, taught, me, great, deal], [was, his, science, officer, during, the, orias, expedition], [leaving, on, mission, to, find, maquis, ship, that, disappeared, in, the, badlands, week, ago], [really, you, ve, never, seen, voyager], [we, like, you, to, come, along], [you, know, the, territory, better, than, anyone, we, ve, got], [my, chief, of, security, was, on, board, undercover], [he, was, supposed, to, report, in, twice, during, the, last, six, days], [he, didn], [maybe], [that, ship, was, under, the, command, of, another, former, starfleet, officer, named, chakotay], [understand, you, knew, him], [the, two, of, you, didn, get, along, too, well, told], [you, help, us, find, that, ship, we, help, you, at, your, next, outmeet, review], [officially, you, be, starfleet, observer, during, the, mission], [you, ll, be, an, observer], [when, it, over, you, re, cut, loose], [and, she, pregnant, oh, mark], [you, ve, got, to, take, her, home, with, you], [she, with, child], [can, leave, her, in, kennel, while, yes], [thanks, honey], [as, soon, as, approve, these, systems, status, reports], [hey], [you, never, bother, me, except, the, way, love, to, be, bothered], [understand, see, you, in, few, weeks], [oh, mark, go, by, my, house, and, pick, up, the, doggie, bed], [she, ll, be, more, comfortable], [come, in], [gentlemen, welcome, aboard, voyager], [mister, kim, at, ease, before, you, sprain, something], [ensign, despite, starfleet, protocol, don, like, being, addressed, as, sir], [ma, am, is, acceptable, in, crunch, but, prefer, captain], [we, re, getting, ready, to, leave], [let, me, show, you, to, the, bridge], [did, you, have, any, problems, getting, here, mister, paris, my, first, officer, lieutenant, commander, cavit], [ensign, kim, mister, paris], [ensign, kim, this, is, your, station], [would, you, like, to, take, over, it, not, crunch, time, yet, mister, kim], [ll, let, you, know, when], [engage], [janeway, to, paris], [report, to, the, bridge], [we, re, approaching, the, badlands], [the, cardassians, gave, us, the, last, known, heading, of, the, maquis, ship, and, we, have, charts, of, the, plasma, storm, activity, the, day, it, disappeared], [with, little, help, we, might, be, able, to, approximate, its, course], [adjust, our, course, to, match], [the, cardassians, claimed, they, forced, the, maquis, ship, into, plasma, storm, where, it, was, destroyed, but, our, probes, haven, picked, up, any, debris], [we, still, be, able, to, pick, up, resonance, trace, from, the, warp, core], [origin, mister, kim], [on, screen], [analysis], [do, it], [red, alert], [move, us, away, from, it, lieutenant], [full, impulse], [can, we, go, to, warp, brace, for, impact], [report, repair, crews, seal, off, hull, breach, on, deck, fourteen], [bridge, to, sickbay], [doctor, can, you, hear, me, paris, how, stadi, need, better, description, than, that, mister, kim], [can, you, get, the, viewscreen, operational, what, about, on, that, that, array, any, idea, what, those, pulses, are, that, are, coming, from, it, mister, kim, try, hailing, the, array], [secure, all, engineering, systems], [on, my, way], [ensign, get, down, to, sickbay], [see, what, going, on], [mister, rollins, the, bridge, is, yours], [what, the, warp, core, pressure, lock, down, the, magnetic, constrictors], [we, don, have, choice], [we, ve, got, to, get, the, reaction, rate, down, before, we, try, to, seal, it], [unlock, the, magnetic, constrictors], [pressure, what, kind, of, scan, bridge, janeway, to, bridge, respond], [initiate, emergency, lock, off], [don, believe, your, eyes, mister, paris], [we, ve, only, transpo

#### Create bigram and trigrams

In [21]:
# def make_bi_and_trigram_models(scripts):
    
#     sentences = scripts[column_name].iloc[0]
    
#     # Build bigram and trigrams

#     bigram = gensim.models.Phrases(sentences, min_count=5, threshold=100)

#     trigram = gensim.models.Phrases(bigram[sentences], threshold=100)  
    
#     # Club sentences as a trigram/bigram

#     bigram_mod = gensim.models.phrases.Phraser(bigram)

#     trigram_mod = gensim.models.phrases.Phraser(trigram)
    
#     result = trigram_mod[bigram_mod[sentences]]
                         
#     return result, bigram_mod, trigram_mod

#### Remove Stopwords, Make Bigrams and Lemmatize

In [22]:
stop_words = stopwords.words('english')

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [23]:
# Define functions for stopwords, bigrams, trigrams, lemmatization

def remove_stopwords(scripts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in scripts]

In [24]:
def make_bi_and_trigram_models(scripts):
    
    sentences = scripts.iloc[0] 
    
    bigram = gensim.models.Phrases(sentences, min_count=5, threshold=100)
    
    trigram = gensim.models.Phrases(bigram[sentences], threshold=100)
    
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    result = trigram_mod[bigram_mod[sentences]]
    
    return result, bigram_mod, trigram_mod

In [25]:
def make_bigrams(scripts, bigram_mod):
    
    return [bigram_mod[doc] for doc in scripts]

In [26]:
def make_trigrams(scripts):
    
    return [trigram_mod[bigram_mod[doc]] for doc in scripts]

In [27]:
def lemmatization(scripts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    """https://spacy.io/api/annotation"""
    
    scripts_out = []
    
    for dialogue in scripts:
        
        doc = nlp(" ".join(dialogue)) 
        
        scripts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    
    return scripts_out

In [28]:
def produce_lemmatized_data(df, column_name='Tokenized Dialogue'):
    # Build bi and trigram models
    result, bigram_mod, trigram_mod = make_bi_and_trigram_models(df[column_name])
    
    # Remove stopwords
    scripts_stopwords_removed = remove_stopwords(result)
    
    # Make bigrams
    scripts_to_bigrams = make_bigrams(scripts_stopwords_removed, bigram_mod)
    
    # Lemmatization
    scripts_lemmatized = lemmatization(scripts_to_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    return scripts_lemmatized

In [29]:
lemmatized_data_picard = produce_lemmatized_data(tokenized_picard_df)

In [30]:
lemmatized_data_picard[:2]

[['agree', 'data', 'starfleet', 'order', 'difficult', 'simple'],
 ['hardly',
  'simple',
  'datum',
  'negotiate',
  'friendly',
  'agreement',
  'starfleet',
  'use',
  'base',
  'time',
  'snoop',
  'find',
  'life_form',
  'build']]

In [31]:
lemmatized_data_janeway = produce_lemmatized_data(tokenized_janeway_df)

In [32]:
lemmatized_data_janeway[:3]

[['kathryn_janeway'], ['serve'], ['wonder', 'go', 'somewhere', 'talk']]

#### Create the Dictionary and Corpus for Topic Modeling

In [36]:
def build_lda_model(df, output_prefix='', num_topics=20, passes=10):
    data_lemmatized = df
    
    # Create dictionary
    id2word = corpora.Dictionary(data_lemmatized)
    
    # Create corpus
    texts = data_lemmatized
    
    # Term document frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    
    # Human-readable term frequency representation
    first_document_representation = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]
    
    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=num_topics,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=passes,
                                                alpha='auto',
                                                per_word_topics=True)
    
    # Get document topics
    doc_lda = lda_model[corpus]
    
    # Prepend the output_prefix to each result
    output_dict = {
        output_prefix + 'id2word': id2word,
        output_prefix + 'corpus': corpus,
        output_prefix + 'first_document_representation': first_document_representation,
        output_prefix + 'lda_model': lda_model,
        output_prefix + 'doc_lda': doc_lda
    }
    
    return output_dict

##### Picard Results

In [55]:
result_dict_picard = build_lda_model(lemmatized_data_picard, 'picard_')

In [56]:
picard_lda_model = result_dict_picard['picard_lda_model']

In [57]:
picard_corpus = result_dict_picard['picard_corpus']

In [58]:
picard_id2word = result_dict_picard['picard_id2word']

In [70]:
picard_lda = picard_lda_model[picard_corpus]

In [60]:
# Compute perplexity. Lower score (below 150 is good, below 50 is better) = better model

print('\nPerplexity: ', picard_lda_model.log_perplexity(picard_lda_corpus))


Perplexity:  -22.516063427097563


In [61]:
# Compute coherence score. Higher score (above .5) = better

picard_coherence_model_lda = CoherenceModel(model=picard_lda_model, texts=lemmatized_data_picard, dictionary=picard_id2word, coherence='c_v')

picard_coherence_lda = picard_coherence_model_lda.get_coherence()

In [62]:
print('\nCoherence Score: ', picard_coherence_lda) # Outcome is a 'fair score'


Coherence Score:  0.4240635506422012


In [63]:
pyLDAvis.enable_notebook()
picard_vis = pyLDAvis.gensim_models.prepare(picard_lda_model, picard_corpus, picard_id2word)

In [64]:
picard_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.172705  0.325178       1        1  10.764061
8      0.325685 -0.245887       2        1  10.059379
7      0.018788  0.004843       3        1   7.687967
9      0.087023  0.049027       4        1   6.927854
17     0.096742  0.059417       5        1   6.797171
13    -0.000138 -0.002264       6        1   6.051634
2      0.007482  0.000357       7        1   5.855653
0     -0.032180 -0.011477       8        1   5.413749
1      0.035319  0.012473       9        1   4.841320
12    -0.043529 -0.014108      10        1   4.831584
6     -0.053786 -0.016159      11        1   4.296994
5     -0.033371 -0.011741      12        1   4.228879
10    -0.068171 -0.018519      13        1   3.859569
14    -0.075733 -0.019604      14        1   3.685954
16    -0.040170 -0.013370      15        1   3.662400
15    -0.075491 -0.019507      16        1   2.935113
18    -0.061640 -0.017423      17        1   2.528564
19    -0.086899 -0.020450      18        1   2.184115
4     -0.085803 -0.020370      19        1   1.948390
11    -0.086833 -0.020415      20        1   1.439649, topic_info=           Term         Freq        Total Category  logprob  loglift
21         know  1846.000000  1846.000000  Default  30.0000  30.0000
16         time  1607.000000  1607.000000  Default  29.0000  29.0000
32           go  1912.000000  1912.000000  Default  28.0000  28.0000
9         datum  1618.000000  1618.000000  Default  27.0000  27.0000
151        ship  1686.000000  1686.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
367   important     0.064336     2.179454  Topic20  -9.4017   0.7181
399        want     0.064372     2.181861  Topic20  -9.4012   0.7175
959       cause     0.064340     2.179945  Topic20  -9.4016   0.7179
1976      large     0.064336     2.179697  Topic20  -9.4017   0.7179
517       right     0.064337     2.180493  Topic20  -9.4017   0.7176

[768 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
221       7  0.958707     accept
0        16  0.990116      agree
6         9  0.953164  agreement
163       2  0.883172      ahead
268      11  0.845739      alive
...     ...       ...        ...
173      18  0.974814       wait
105      15  0.992571       warp
253       6  0.997975        way
116       5  0.996342       well
27       18  0.955789       word

[164 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 8, 10, 18, 14, 3, 1, 2, 13, 7, 6, 11, 15, 17, 16, 19, 20, 5, 12])

##### Janeway Results

In [65]:
result_dict_janeway = build_lda_model(lemmatized_data_janeway, 'janeway_')

In [66]:
janeway_lda_model = result_dict_janeway['janeway_lda_model']

In [67]:
janeway_corpus = result_dict_janeway['janeway_corpus']

In [68]:
janeway_id2word = result_dict_janeway['janeway_id2word']

In [71]:
janeway_lda = janeway_lda_model[janeway_corpus]

In [73]:
# Compute perplexity. Lower score (below 150 is good, below 50 is better) = better model

print('\nPerplexity: ', janeway_lda_model.log_perplexity(janeway_corpus))


Perplexity:  -21.871265173547325


In [74]:
# Compute coherence score. Higher score (above .5) = better

janeway_coherence_model_lda = CoherenceModel(model=janeway_lda_model, texts=lemmatized_data_janeway, dictionary=janeway_id2word, coherence='c_v')

janeway_coherence_lda = janeway_coherence_model_lda.get_coherence()

In [75]:
print('\nCoherence Score: ', janeway_coherence_lda) # Outcome is a 'fair score'


Coherence Score:  0.44456827700081014


In [76]:
pyLDAvis.enable_notebook()
janeway_vis = pyLDAvis.gensim_models.prepare(janeway_lda_model, janeway_corpus, janeway_id2word)

In [77]:
janeway_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.252366 -0.292833       1        1  10.347235
18     0.216126  0.300918       2        1   8.550651
13     0.083990  0.011205       3        1   8.529691
16     0.108927  0.018076       4        1   7.690010
3      0.041612  0.003841       5        1   6.819738
14     0.029665  0.002340       6        1   6.193760
12     0.096900  0.014400       7        1   5.951187
10    -0.030312 -0.003049       8        1   5.777282
0     -0.002224 -0.000891       9        1   5.323911
8     -0.028388 -0.002854      10        1   5.058222
17    -0.018426 -0.002187      11        1   5.008460
15    -0.062825 -0.004788      12        1   4.364414
19    -0.073064 -0.005225      13        1   3.802492
7     -0.075245 -0.005300      14        1   3.664515
2     -0.072001 -0.005152      15        1   3.162194
4     -0.088770 -0.005703      16        1   3.122004
9     -0.085434 -0.005616      17        1   2.564826
11    -0.090368 -0.005751      18        1   2.003358
5     -0.096350 -0.005826      19        1   1.406205
6     -0.106179 -0.005605      20        1   0.659844, topic_info=                Term         Freq        Total Category  logprob  loglift
34               get  2495.000000  2495.000000  Default  30.0000  30.0000
2                 go  2050.000000  2050.000000  Default  29.0000  29.0000
35              know  1654.000000  1654.000000  Default  28.0000  28.0000
61              take  1367.000000  1367.000000  Default  27.0000  27.0000
189            think  1389.000000  1389.000000  Default  26.0000  26.0000
...              ...          ...          ...      ...      ...      ...
5111  understatement     0.050587     2.226485  Topic20  -8.8398   1.2364
1924           equip     0.050587     2.226469  Topic20  -8.8398   1.2364
280            tuvok     0.050593     2.227020  Topic20  -8.8397   1.2363
4908           icheb     0.050589     2.226705  Topic20  -8.8398   1.2364
579             feel     0.050587     2.226995  Topic20  -8.8398   1.2362

[758 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
105       6  0.988271         able
177       9  0.958766  acknowledge
99        8  0.960299     activity
109       7  0.980946       adjust
21       11  0.986907          ago
...     ...       ...          ...
29       12  0.986674         week
76        5  0.984052      welcome
37       16  0.995285         well
10        2  0.995426         work
96        4  0.989102          yet

[137 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 19, 14, 17, 4, 15, 13, 11, 1, 9, 18, 16, 20, 8, 3, 5, 10, 12, 6, 7])